# Data loader

In [1]:
# TODO 
# Load data from REST API onto disk -- only DL the new data

In [1]:
from entsoe import EntsoePandasClient

In [2]:
client = EntsoePandasClient(api_key=<YOUR API KEY>)

start = pd.Timestamp('20171201', tz='Europe/Brussels')
end = pd.Timestamp('20180101', tz='Europe/Brussels')
country_code = 'CH' 
type_marketagreement_type = 'A01'
contract_marketagreement_type = "A01"
process_type = 'A51'

SyntaxError: invalid syntax (3218539740.py, line 1)

In [ ]:
client.query_load_and_forecast(country_code, start=start, end=end)